In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import requests
import json
from tqdm import tqdm
import time
from datetime import datetime
import os
import glob
import pickle
from typing import Dict, List, Optional, Tuple


In [2]:
# Configuración de API
OPENROUTER_API_KEY = "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"  # Obtener de https://openrouter.ai/keys
OPENROUTER_API_URL = "https://openrouter.ai/api/v1/chat/completions"

# Modelo recomendado (puedes cambiar según necesidades y presupuesto)
MODEL = "google/gemini-2.5-flash"  # Rápido y económico
# Alternativas: "openai/gpt-3.5-turbo", "meta-llama/llama-3-8b-instruct"

# Headers para las peticiones
headers = {
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
    "Content-Type": "application/json",
    "HTTP-Referer": "http://localhost:8888",  # Requerido por OpenRouter
    "X-Title": "TrustPilot Analysis"
}

# Definir los campos esperados en la respuesta del modelo (en orden)
CAMPOS_ANALISIS = [
    "language", "sentiment", "sentiment_score", "emotion", "emotion_intensity", "customer_gender",
    "main_topic", "keywords", "customer_type", "tourist_type", "group_type"
]

# Variables globales para los nombres de las columnas (se asignarán dinámicamente)
REVIEW_TEXT_COL = None
CUSTOMER_NAME_COL = None

In [3]:
# Cargar el CSV con las reseñas
df = pd.read_csv('./test.csv', encoding='utf-8-sig')

# Limpiar nombres de columnas (quitar espacios extra)
df.columns = df.columns.str.strip()

# Mostrar todas las columnas para identificar correctamente
print(f"Total de reseñas: {len(df)}")
print(f"Columnas en el CSV:")
for i, col in enumerate(df.columns):
    print(f"  {i}: '{col}' (longitud: {len(col)})")

# Buscar la columna de review_text (puede tener nombre ligeramente diferente)
texto_columnas = [col for col in df.columns if 'review' in col.lower() and 'text' in col.lower()]
nombre_columnas = [col for col in df.columns if 'customer' in col.lower() and 'name' in col.lower()]

print(f"\nColumnas que contienen 'review' y 'text': {texto_columnas}")
print(f"Columnas que contienen 'customer' y 'name': {nombre_columnas}")

# Si encontramos las columnas, proceder con la limpieza
if texto_columnas and nombre_columnas:
    # Asignar variables globales
    REVIEW_TEXT_COL = texto_columnas[0]
    CUSTOMER_NAME_COL = nombre_columnas[0]
    
    print(f"\nUsando columnas:")
    print(f"- Texto de reseña: '{REVIEW_TEXT_COL}'")
    print(f"- Nombre del cliente: '{CUSTOMER_NAME_COL}'")
    
    # Limpiar datos: eliminar filas con review_text vacío o nulo
    df_original_len = len(df)
    df = df.dropna(subset=[REVIEW_TEXT_COL])  # Eliminar filas con texto nulo
    df = df[df[REVIEW_TEXT_COL].astype(str).str.strip() != '']  # Eliminar filas con texto vacío
    
    print(f"\nLimpieza de datos:")
    print(f"- Reseñas originales: {df_original_len}")
    print(f"- Reseñas después de limpieza: {len(df)}")
    print(f"- Reseñas eliminadas: {df_original_len - len(df)}")
    
    # Inicializar la columna 'analyzed' si no existe
    if 'analyzed' not in df.columns:
        df['analyzed'] = False
        
    print(f"- Reseñas ya analizadas: {df['analyzed'].sum()}")
    print(f"- Reseñas pendientes de analizar: {(df['analyzed'] == False).sum()}")
    
else:
    print("\n❌ No se encontraron las columnas de texto de reseña o nombre del cliente")
    print("Por favor revisa el archivo CSV")

Total de reseñas: 80
Columnas en el CSV:
  0: 'review_id' (longitud: 9)
  1: 'domain' (longitud: 6)
  2: 'company_name' (longitud: 12)
  3: 'categories' (longitud: 10)
  4: 'subcategories' (longitud: 13)
  5: 'company_rating' (longitud: 14)
  6: 'review_date' (longitud: 11)
  7: 'customer_name' (longitud: 13)
  8: 'customer_score' (longitud: 14)
  9: 'review_text' (longitud: 11)
  10: 'language' (longitud: 8)
  11: 'sentiment' (longitud: 9)
  12: 'emotion' (longitud: 7)
  13: 'customer_gender' (longitud: 15)
  14: 'main_topic' (longitud: 10)
  15: 'keywords' (longitud: 8)
  16: 'customer_type' (longitud: 13)
  17: 'tourist_type' (longitud: 12)
  18: 'group_type' (longitud: 10)
  19: 'analyzed' (longitud: 8)

Columnas que contienen 'review' y 'text': ['review_text']
Columnas que contienen 'customer' y 'name': ['customer_name']

Usando columnas:
- Texto de reseña: 'review_text'
- Nombre del cliente: 'customer_name'

Limpieza de datos:
- Reseñas originales: 80
- Reseñas después de limpiez

In [4]:
def crear_prompt_analisis(review_text, customer_name):
    """Crea el prompt para analizar una reseña"""
    
    prompt = f"""
Eres un analizador especializado en evaluación de reseñas turísticas y análisis de sentimientos. Para cada texto que recibas, deberás analizar y proporcionar la siguiente información separada por el delimitador "|":

RESEÑA A ANALIZAR:
Texto: {review_text}
Cliente: {customer_name}

ANÁLISIS REQUERIDO (responde cada campo separado por "|"):

0. Language: Clasifica como "es", "en", "fr", "de", "it", "pt", "nl", "ru", "tr", "ar", "zh", "ja", "ko", "other"
1. Sentiment: Clasifica como "Positivo", "Negativo" o "Neutro"
2. Sentiment_score: Evalúa en escala de -1 a +1 (-1=extremadamente negativo, 0=neutro, +1=extremadamente positivo)
3. Emotion: Identifica una emoción (joy, surprise, neutral, sadness, disgust, anger, fear)
4. Emotion_intensity: Intensidad de 1-5 (1=muy leve, 5=muy intensa)
5. Customer_gender: Basado en el nombre (masculino, femenino, unknown)
6. Topic: Tema principal (Atención al cliente, Limpieza, Instalaciones, Relación calidad-precio, Servicios, Ubicación, Ética y sostenibilidad, Check-in y Check-out, Comodidad y descanso, Oferta gastronómica, Facilidad de reserva y accesibilidad digital, Animación y actividades, Seguridad)
7. Keywords: 3-5 términos relevantes separados por comas SIN espacios
8. Customer_type: Promotor, Leal, Neutral, Crítico, Oportunista
9. Tourist_type: Turista de ocio, cultural, naturaleza, aventura, compras, espiritual/religioso, gastronómico, deportivo, wellness, solidario/voluntario
10. Group_type: familiar, amigos, pareja, solitario, grupo organizado

FORMATO DE RESPUESTA:
Responde ÚNICAMENTE con los valores separados por "|" en el orden exacto listado arriba.
Si no puedes determinar algún campo, usa "unknown".
NO incluyas espacios antes o después de los pipes.

Ejemplo: Positivo|0.8|joy|4|femenino|Atención al cliente|excelente,servicio,amable|Promotor|Turista de ocio|pareja
"""
    return prompt

In [5]:
def analizar_con_llm(review_text, customer_name, max_retries=3):
    """Llama a OpenRouter para analizar una reseña"""
    
    prompt = crear_prompt_analisis(review_text, customer_name)
    
    payload = {
        "model": MODEL,
        "messages": [
            {"role": "system", "content": "Eres un experto en análisis de reseñas de viajes. Respondes SOLO con los valores separados por |."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.1,  # Baja temperatura para respuestas consistentes
        "max_tokens": 500
    }
    
    for intento in range(max_retries):
        try:
            response = requests.post(
                OPENROUTER_API_URL,
                headers=headers,
                json=payload,
                timeout=30
            )
            
            if response.status_code == 200:
                result = response.json()
                content = result['choices'][0]['message']['content']
                
                print(f"Respuesta del modelo: {content}")  # Para depuración
                
                if not content.strip():
                    print("Respuesta vacía de la API")
                    return None
                
                # Limpiar posibles markdown o espacios extra
                if "```" in content:
                    # Extraer contenido entre markdown
                    lines = content.split('\n')
                    for line in lines:
                        if '|' in line and not line.strip().startswith('```'):
                            content = line.strip()
                            break
                
                content = content.strip()
                
                # Dividir por el delimitador pipe
                valores = [v.strip() for v in content.split("|")]
                
                # Verificar que tenemos el número correcto de campos
                if len(valores) != len(CAMPOS_ANALISIS):
                    print(f"La respuesta no tiene el número correcto de campos: {len(valores)} vs {len(CAMPOS_ANALISIS)}")
                    print(f"Valores recibidos: {valores}")
                    return None
                
                # Crear diccionario con los resultados
                resultado = dict(zip(CAMPOS_ANALISIS, valores))
                return resultado
            
            elif response.status_code == 429:  # Rate limit
                time.sleep(2 ** intento)  # Exponential backoff
                continue
            
            else:
                print(f"Error API: {response.status_code} - {response.text}")
                return None
                
        except Exception as e:
            print(f"Error en petición: {e}")
            if intento < max_retries - 1:
                time.sleep(1)
    
    return None

In [6]:
def procesar_reseñas_batch(df, batch_size=10, start_index=0):
    """Procesa las reseñas en lotes para evitar límites de rate"""
    
    global REVIEW_TEXT_COL, CUSTOMER_NAME_COL
    
    if REVIEW_TEXT_COL is None or CUSTOMER_NAME_COL is None:
        print("❌ Error: Las columnas de texto y nombre no han sido identificadas")
        return [], []
    
    # Filtrar solo reseñas no analizadas (ya limpiamos los datos vacíos en la carga)
    df_pendientes = df[df['analyzed'] == False].iloc[start_index:]
    
    print(f"Reseñas pendientes de analizar: {len(df_pendientes)}")
    
    resultados = []
    errores = []
    
    # Procesar en lotes
    for i in tqdm(range(0, len(df_pendientes), batch_size), desc="Procesando lotes"):
        batch = df_pendientes.iloc[i:i+batch_size]
        
        for idx, row in batch.iterrows():
            # Analizar reseña usando nombres de columnas dinámicos
            resultado = analizar_con_llm(
                row[REVIEW_TEXT_COL], 
                row[CUSTOMER_NAME_COL]
            )
            
            if resultado:
                resultado['index'] = idx
                resultados.append(resultado)
            else:
                errores.append({
                    'index': idx,
                    'review_id': row.get('review_id', 'N/A'),
                    'error': 'No se pudo analizar'
                })
            
            # Pequeña pausa entre peticiones
            time.sleep(0.5)
        
        # Pausa más larga entre lotes
        print(f"Lote completado. Esperando antes del siguiente...")
        time.sleep(2)
    
    return resultados, errores

In [7]:
def actualizar_dataframe(df, resultados):
    """Actualiza el DataFrame con los resultados del análisis"""
    
    # Inicializar las columnas si no existen
    for campo in CAMPOS_ANALISIS:
        if campo not in df.columns:
            df[campo] = None
    
    # Agregar columna 'analyzed' si no existe
    if 'analyzed' not in df.columns:
        df['analyzed'] = False
    
    # Actualizar con los resultados
    for resultado in resultados:
        idx = resultado['index']
        
        # Actualizar cada campo
        for campo in CAMPOS_ANALISIS:
            if campo in resultado:
                df.loc[idx, campo] = resultado[campo]
        
        # Marcar como analizado
        df.loc[idx, 'analyzed'] = True
    
    return df


In [8]:
def guardar_progreso(df, filename_base='trustpilot_analyzed'):
    """Guarda el progreso del análisis"""
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{filename_base}_{timestamp}.csv"
    
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"Progreso guardado en: {filename}")
    
    # También guardar un backup del último estado
    df.to_csv(f"{filename_base}_latest.csv", index=False, encoding='utf-8-sig')
    
    return filename

In [9]:
def analizar_trustpilot_reviews(csv_path, batch_size=10, max_reviews=None):
    """Función principal para analizar las reseñas"""
    
    # Cargar datos
    print("📂 Cargando datos...")
    df = pd.read_csv(csv_path, encoding='utf-8-sig')
    
    # Limitar número de reseñas si se especifica
    if max_reviews:
        df = df.head(max_reviews)
    
    print(f"📊 Total de reseñas a procesar: {len(df)}")
    
    # Verificar API key
    if not OPENROUTER_API_KEY or OPENROUTER_API_KEY == "tu-api-key-aqui":
        print("❌ Error: Configura tu API key de OpenRouter")
        return None
    
    # Procesar reseñas
    print("\n🤖 Iniciando análisis con LLM...")
    resultados, errores = procesar_reseñas_batch(df, batch_size)
    
    # Actualizar DataFrame
    print(f"\n✅ Análisis completado: {len(resultados)} reseñas")
    print(f"❌ Errores: {len(errores)} reseñas")
    
    df_actualizado = actualizar_dataframe(df, resultados)
    
    # Guardar resultados
    filename = guardar_progreso(df_actualizado)
    
    # Mostrar estadísticas
    print("\n📈 Estadísticas del análisis:")
    if 'sentiment' in df_actualizado.columns:
        print(f"- Sentimientos: {df_actualizado['sentiment'].value_counts().to_dict()}")
    if 'main_topic' in df_actualizado.columns:
        print(f"- Temas principales: {df_actualizado['main_topic'].value_counts().to_dict()}")
    if 'tourist_type' in df_actualizado.columns:
        print(f"- Tipos de turista: {df_actualizado['tourist_type'].value_counts().to_dict()}")
    if 'emotion' in df_actualizado.columns:
        print(f"- Emociones: {df_actualizado['emotion'].value_counts().to_dict()}")
    
    return df_actualizado, errores

In [10]:
# Configurar parámetros
CSV_PATH = "./test.csv"  # Ajusta el nombre
BATCH_SIZE = 10  # Reseñas por lote
MAX_REVIEWS = 100  # None para procesar todas

# Ejecutar análisis
df_analizado, errores = analizar_trustpilot_reviews(
    csv_path=CSV_PATH,
    batch_size=BATCH_SIZE,
    max_reviews=MAX_REVIEWS
)

📂 Cargando datos...
📊 Total de reseñas a procesar: 80

🤖 Iniciando análisis con LLM...
Reseñas pendientes de analizar: 80


Procesando lotes:   0%|          | 0/8 [00:00<?, ?it/s]

Respuesta del modelo: es|Positivo|0.95|joy|5|masculino|Servicios|Japón,sueños,guia,Zaida,viaje|Promotor|cultural|grupoorganizado
Respuesta del modelo: es|Positivo|0.9|joy|4|femenino|Servicios|Japón,programa,agencia,recomiendo|Promotor|cultural|grupoorganizado
Respuesta del modelo: es|Positivo|0.8|joy|4|masculino|Servicios|guia,lugares,caminar,adelgazar,recomendable|Promotor|Turista cultural|grupo organizado
Respuesta del modelo: es|Positivo|0.9|joy|4|femenino|Oferta gastronómica|sitiosincreíbles,ruta,Zaida|Promotor|cultural,gastronómico|unknown
Respuesta del modelo: es|Positivo|0.9|joy|5|masculino|Servicios|vuelos,traslados,hoteles,excursiones,guía|Promotor|Turista de ocio|grupo organizado
Respuesta del modelo: es|Positivo|0.9|joy|4|femenino|Servicios|transporte,guia,ruta,pais,encanto|Promotor|Turista cultural|unknown
Respuesta del modelo: es|Positivo|0.9|joy|5|masculino|Animación y actividades|organización,naturaleza,guía,excepcional,libertad|Promotor|Turista de naturaleza|grupo organ

Procesando lotes:  12%|█▎        | 1/8 [00:31<03:37, 31.00s/it]

Respuesta del modelo: es|Positivo|0.95|joy|5|unknown|Servicios|Japón,tour,guía,profesional,recomendadísimo|Promotor|cultural|grupo organizado
Respuesta del modelo: es|Positivo|0.7|joy|3|masculino|unknown|viaje,completo|Promotor|Turista de ocio|unknown
Respuesta del modelo: es|Positivo|0.9|joy|5|femenino|Servicios|organizado,desplazamientos,guías,amables,eficientes|Promotor|Turista de ocio|grupo organizado
Respuesta del modelo: es|Positivo|0.9|joy|4|masculino|Servicios|organizado,DescubriendoJapón,bien|Promotor|cultural|unknown
Respuesta del modelo: es|Positivo|0.9|joy|4|femenino|Atención al cliente|fantástico,organizado,esfuerzo,dedicación,Josep|Promotor|Turista de ocio|grupo organizado
Respuesta del modelo: es|Positivo|0.9|joy|4|unknown|Servicios|viaje,organizado,Japón,diferente|Promotor|cultural|unknown
Respuesta del modelo: es|Positivo|0.9|joy|5|femenino|Servicios|experiencia,organización,guías,recomiendo,fenomenal|Promotor|Turista de ocio|unknown
Respuesta del modelo: es|Neutro|0|n

...

Procesando lotes:  88%|████████▊ | 7/8 [03:05<00:24, 24.43s/it]

Respuesta del modelo: es|Positivo|0.9|joy|5|femenino|Atención al cliente|recorrido,guía,profesional,divertida,recomendable|Promotor|Turista de ocio|unknown
Respuesta del modelo: es|Positivo|0.9|joy|4|masculino|Ética y sostenibilidad|profesionalidad,transparencia,honradez,guía|Promotor|Turista cultural|solitario
Respuesta del modelo: es|Positivo|0.9|joy|4|masculino|Servicios|guía,informada,entretenida,recomendar|Promotor|Turista cultural|unknown
Respuesta del modelo: es|Positivo|0.9|joy|4|femenino|Servicios|CascoViejo,Ensanche,Francisco,historia,arquitectura|Promotor|cultural|unknown
Respuesta del modelo: es|Positivo|0.9|joy|4|unknown|Animación y actividades|ciudad,Mariela,daneses,información,interesante|Promotor|cultural|unknown
Respuesta del modelo: es|Positivo|0.9|joy|5|femenino|Atención al cliente|guía,ameno,conocimiento,Maurizio,vacaciones|Promotor|Turista de ocio|unknown
Respuesta del modelo: es|Positivo|0.9|joy|4|masculino|Servicios|fácil,gestionar,comunicación,guías,gusta|Promot

Procesando lotes: 100%|██████████| 8/8 [03:25<00:00, 25.69s/it]


✅ Análisis completado: 74 reseñas
❌ Errores: 6 reseñas
Progreso guardado en: trustpilot_analyzed_20250702_124127.csv

📈 Estadísticas del análisis:
- Sentimientos: {'Positivo': 64, 'Neutro': 7, 'unknown': 2, 'Negativo': 1}
- Temas principales: {'Servicios': 18, 'Atención al cliente': 14, 'unknown': 13, 'Relación calidad-precio': 11, 'Facilidad de reserva y accesibilidad digital': 8, 'Animación y actividades': 3, 'Atenciónalcliente': 3, 'Oferta gastronómica': 2, 'Instalaciones': 1, 'Ética y sostenibilidad': 1}
- Tipos de turista: {'Turista de ocio': 33, 'unknown': 16, 'cultural': 13, 'Turista cultural': 8, 'cultural,gastronómico': 1, 'Turista de naturaleza': 1, 'Turistadeocio': 1, 'Turistacultural': 1}
- Emociones: {'joy': 63, 'neutral': 8, 'unknown': 2, 'anger': 1}



/var/folders/2h/j25t7r2x2gz01h_zbbny8klh0000gn/T/ipykernel_12181/2484441819.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'es' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[idx, campo] = resultado[campo]
/var/folders/2h/j25t7r2x2gz01h_zbbny8klh0000gn/T/ipykernel_12181/2484441819.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Positivo' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[idx, campo] = resultado[campo]
/var/folders/2h/j25t7r2x2gz01h_zbbny8klh0000gn/T/ipykernel_12181/2484441819.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'joy' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[idx, campo] = resulta

In [11]:
'''
# Crear visualizaciones básicas
import matplotlib.pyplot as plt
import seaborn as sns

# Solo ejecutar si df_analizado existe y tiene datos analizados
if 'df_analizado' in locals() and df_analizado is not None:
    # Configurar estilo
    plt.style.use('seaborn-v0_8')
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # 1. Distribución de sentimientos
    if 'sentiment' in df_analizado.columns:
        df_analizado['sentiment'].value_counts().plot(kind='bar', ax=axes[0,0])
        axes[0,0].set_title('Distribución de Sentimientos')
    
    # 2. Top 10 temas principales
    if 'main_topic' in df_analizado.columns:
        df_analizado['main_topic'].value_counts().head(10).plot(kind='barh', ax=axes[0,1])
        axes[0,1].set_title('Top 10 Temas Principales')
    
    # 3. Tipos de turista
    if 'tourist_type' in df_analizado.columns:
        df_analizado['tourist_type'].value_counts().plot(kind='pie', ax=axes[1,0])
        axes[1,0].set_title('Tipos de Turista')
    
    # 4. Emociones detectadas
    if 'emotion' in df_analizado.columns:
        df_analizado['emotion'].value_counts().plot(kind='bar', ax=axes[1,1])
        axes[1,1].set_title('Emociones Detectadas')

    plt.tight_layout()
    plt.show()
else:
    print("No hay datos analizados para visualizar. Ejecuta primero el análisis.")

'''

'\n# Crear visualizaciones básicas\nimport matplotlib.pyplot as plt\nimport seaborn as sns\n\n# Solo ejecutar si df_analizado existe y tiene datos analizados\nif \'df_analizado\' in locals() and df_analizado is not None:\n    # Configurar estilo\n    plt.style.use(\'seaborn-v0_8\')\n    fig, axes = plt.subplots(2, 2, figsize=(15, 10))\n\n    # 1. Distribución de sentimientos\n    if \'sentiment\' in df_analizado.columns:\n        df_analizado[\'sentiment\'].value_counts().plot(kind=\'bar\', ax=axes[0,0])\n        axes[0,0].set_title(\'Distribución de Sentimientos\')\n    \n    # 2. Top 10 temas principales\n    if \'main_topic\' in df_analizado.columns:\n        df_analizado[\'main_topic\'].value_counts().head(10).plot(kind=\'barh\', ax=axes[0,1])\n        axes[0,1].set_title(\'Top 10 Temas Principales\')\n    \n    # 3. Tipos de turista\n    if \'tourist_type\' in df_analizado.columns:\n        df_analizado[\'tourist_type\'].value_counts().plot(kind=\'pie\', ax=axes[1,0])\n        axe